In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import socket

In [2]:
LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [45]:
spark = SparkSession\
    .builder\
    .master("k8s://https://10.32.7.103:6443")\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "1")\
    .config("spark.executor.cores", '1')\
    .config("spark.executor.memory", "4g")\
    .config("spark.kubernetes.namespace", "mrjazanov-338247")\
    .config("spark.kubernetes.container.image.pullPolicy", "Always")\
    .config("spark.kubernetes.container.image", "node03.st:5000/spark-executor:mrjazanov-338247")\
    .config("spark.kubernetes.executor.deleteOnTermination", "true")\
    .config("spark.driver.extraClassPath","./clickhouse-native-jdbc-shaded-2.6.5.jar")\
    .config("spark.executor.extraClassPath", "./clickhouse-native-jdbc-shaded-2.6.5.jar")\
    .config("spark.jars", "./clickhouse-native-jdbc-shaded-2.6.5.jar")\
    .getOrCreate()

In [46]:
spark

In [50]:
url="jdbc:clickhouse://clickhouse-0.clickhouse.clickhouse"
user="mrjazanov_338247" #replace by whatever you use
password="6TGFWHEOeX" #same here
dbtable='mrjazanov_338247.distr_transactions'
driver="com.github.housepower.jdbc.ClickHouseDriver"

In [51]:
df = spark.read.format('jdbc').option('driver',driver).option('url',url).option('user',user).option('password',password).option('dbtable',dbtable).load()

In [53]:
df.select('user_id_out').where(F.col('user_id_out') > 1500).limit(10).show()

+-----------+
|user_id_out|
+-----------+
|       7526|
|       7526|
|       7526|
|       7526|
|       7526|
|       7526|
|       7526|
|       7526|
|       7526|
|       7526|
+-----------+



In [54]:
spark.stop()